In [9]:
import os    
import pandas as pd
import ollama
import chromadb

In [10]:
def parse(text):
    abs = text.split("\\\\")[2].strip()
    auth = text.split("\\\\")[1].strip().split("Comment")[0].split(":")[-1]
    return abs,auth


In [11]:
ids = []
abstracts = []
authors = []

In [12]:
for folder in os.listdir('cit-HepTh-abstracts'):
    if folder==".DS_Store":
        continue
    for file in os.listdir('cit-HepTh-abstracts/'+folder):
        with open('cit-HepTh-abstracts/'+folder+'/'+file) as f:
            ids.append(file.split(".")[0])
            abstract, author = parse(f.read())
            abstracts.append(abstract)
            authors.append(author)


FileNotFoundError: [Errno 2] No such file or directory: 'cit-HepTh-abstracts'

In [7]:
print(authors[:100])

[' M{\\aa}ns Henningson and Chiara R. Nappi\n', ' Abbas Ali and Alok Kumar\n', ' Phys.Lett. B302 (1993) 396-402', ' K. Demeterfi and I. R. Klebanov\n', ' F.W. Nijhoff and H.W. Capel\n', ' Joseph A. Minahan\n', ' I. Antoniadis, P.O. Mazur and E. Mottola\n', ' G.K.Savvidy and K.G.Savvidy\n', ' Yi-Zhi Huang and James Lepowsky\n', ' Donald Spector\n', ' G. Bonneau, G. Valent\n', ' R. Jackiw, Q. Liu and C. Lucchesi\n', ' Jorge Ananias Neto\n', ' R. Brooks and C. Lucchesi\n', ' S. Kharchev, A.Marshakov\n', ' Wolfram Boenkost\n', ' T. Curtright and C. Zachos\n', ' A. Anderson, B.E.W. Nilsson, C.N. Pope and K.S. Stelle\n', ' S. De Leo and P. Rotelli\n', ' Subir Ghoshal\n', ' Robert Marnelius,Ulrich Quaade\n', ' M.A. de Andrade and O.M. Del Cima\n', ' Fuad Saradzhev\n', ' M. R\\"osgen, R. Varnhagen\n', ' Igor Batalin,Robert Marnelius\n', ' J.Karkowski, Z.Swierczynski, P.Wegrzyn\n', " Zala'n Horva'th and Ga'bor Taka'cs\n", ' Geoffrey Dixon\n', ' Pawel Wegrzyn\n', ' Ramesh Abhiraman and Charles M

In [72]:
client = chromadb.PersistentClient(path="full_data/")
collection = client.get_collection(name="authors")

# store each document in a vector embedding database
for i, d in enumerate(authors):
  response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[ids[i]],
    embeddings=[embedding],
    documents=[d]
  )
  


In [37]:
client = chromadb.PersistentClient(path="full_data/")
collection = client.create_collection(name="docs")

# store each document in a vector embedding database
for i, d in enumerate(abstracts):
  response = ollama.embeddings(model="mxbai-embed-large", prompt=d)
  embedding = response["embedding"]
  collection.add(
    ids=[ids[i]],
    embeddings=[embedding],
    documents=[d]
  )
  


In [8]:
len(abstracts)

119

In [21]:
client = chromadb.PersistentClient(path="full_data/")
collection = client.get_collection(name="docs")

# an example prompt
prompt = """  Effective world-brane actions for solitons of ten-dimensional type IIA and
IIB superstring theory are derived using the formulation of solitons as
Dirichlet branes. The one-brane actions are used to recover predictions of
SL(2,Z) strong-weak coupling duality. The two-brane action, which contains a
hidden eleventh target space coordinate, is shown to be the eleven-dimensional
supermembrane action. It can be thought of as the membrane action of
`M-theory'."""

# generate an embedding for the prompt and retrieve the most relevant doc
response = ollama.embeddings(
  prompt=prompt,
  model="mxbai-embed-large"
)
results = collection.query(
  query_embeddings=[response["embedding"]],
  n_results=10,
)
data = results['documents'][0][0]
data

"Effective world-brane actions for solitons of ten-dimensional type IIA and\nIIB superstring theory are derived using the formulation of solitons as\nDirichlet branes. The one-brane actions are used to recover predictions of\nSL(2,Z) strong-weak coupling duality. The two-brane action, which contains a\nhidden eleventh target space coordinate, is shown to be the eleven-dimensional\nsupermembrane action. It can be thought of as the membrane action of\n`M-theory'."

In [24]:
results

{'ids': [['9601003',
   '9603116',
   '9609004',
   '9804076',
   '0301223',
   '9512062',
   '9703118',
   '9804051',
   '9501068',
   '9602112']],
 'embeddings': None,
 'documents': [["Effective world-brane actions for solitons of ten-dimensional type IIA and\nIIB superstring theory are derived using the formulation of solitons as\nDirichlet branes. The one-brane actions are used to recover predictions of\nSL(2,Z) strong-weak coupling duality. The two-brane action, which contains a\nhidden eleventh target space coordinate, is shown to be the eleven-dimensional\nsupermembrane action. It can be thought of as the membrane action of\n`M-theory'.",
   "String and membrane dynamics may be unified into a theory of 2+2 dimensional\nself-dual world-volumes living in a 10+2 dimensional target space. Some of the\nvacua of this M-theory are described by the N=(2,1) heterotic string, whose\ntarget space theory describes the world-volume dynamics of 2+2 dimensional\n`M-branes'. All classes of stri